In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/cleaned_datasets/train.csv')
eval = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/cleaned_datasets/eval.csv')
test = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/cleaned_datasets/test.csv')

In [ ]:
jobs_score_mat = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/score_matrices/scoring_matrix_jobs.csv')
implicit_score_mat = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/score_matrices/scoring_matrix_implicit.csv')
item_cf_score_mat = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/score_matrices/scoring_matrix_item_based_cf.csv')
ncf_score_mat = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/score_matrices/scoring_matrix_ncf.csv')
lightgcn_score_mat = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/score_matrices/scoring_matrix_lightgcn.csv')
cb_score_mat = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/score_matrices/scoring_matrix_content_based.csv')
user_cf_score_mat = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/score_matrices/scoring_matrix_ubcf.csv')

### Get 1000 selected reviewer names

In [ ]:
reviewer_names = jobs_score_mat.iloc[:, 0].tolist()
print(reviewer_names)

['shuying x', 'patricia g', 'matt h', 'allison w', 'joacim n', 'min m t', 'kane s', 'top g', 'ashok k', 'abdelkader b', 'soumik c', 'jatin p', 'mary s', 'lauren k', 'adriana p', 'trevor s', 'gino e n', 'raju', 'greg v', 'omkar j', 'ludmila r', 'patrick a', 'hubert s', 'sergio b', 'hamid k', 'tng n t', 'sonali p', 'donald b', 'john a v g', 'georgios l', 'neeraj', 'bhavesh p', 'serena r t', 'sergei t', 'lawrence', 'sudheer d', 'amit k a', 'olga l', 'tyler p', 'ankit t', 'hermes r s a', 'zhe w', 'husam k m', 'tony m', 'gilbert c', 'maria s c', 'matt o', 'mohammad y', 'mahmoud m', 'anna k', 'ishan b', 'nguyen h t', 'fernanda s', 'angel l k', 'brandon l', 'mintu s', 'sergey k', 'jorge l l u', 'priyanshu a', 'daniel p c', 'maulik s', 'rishikesh p', 'manthan s', 'sridhar m', 'victor k o', 'victoria', 'nnamdi o', 'debneil s r', 'alexandra p', 'sandra k', 'alicia', 'omar s m', 'karthick s', 'melanie n', 'yang d', 'zahir k', 'miguel a b', 'r c', 'ethan l', 'richard m', 'vladimir n', 'abdulrahman

### Remove other reviewers that we are not making recommendations for

In [ ]:
final_implicit_score_mat = implicit_score_mat[implicit_score_mat.iloc[:, 0].isin(reviewer_names)].reset_index(drop=True)
final_item_cf_score_mat = item_cf_score_mat[item_cf_score_mat.iloc[:, 0].isin(reviewer_names)].reset_index(drop=True)
final_lightgcn_score_mat = lightgcn_score_mat[lightgcn_score_mat.iloc[:, 0].isin(reviewer_names)].reset_index(drop=True)
final_ncf_score_mat = ncf_score_mat[ncf_score_mat.iloc[:, 0].isin(reviewer_names)].reset_index(drop=True)
final_jobs_score_mat = jobs_score_mat[jobs_score_mat.iloc[:, 0].isin(reviewer_names)].reset_index(drop=True)
final_cb_score_mat = cb_score_mat[cb_score_mat.iloc[:, 0].isin(reviewer_names)].reset_index(drop=True)
final_user_cf_score_mat = user_cf_score_mat[user_cf_score_mat.iloc[:, 0].isin(reviewer_names)].reset_index(drop=True)

In [ ]:
print(final_implicit_score_mat.shape)
print(final_item_cf_score_mat.shape)
print(final_lightgcn_score_mat.shape)
print(final_ncf_score_mat.shape)
print(final_jobs_score_mat.shape)
print(final_cb_score_mat.shape)
print(final_user_cf_score_mat.shape)

(1000, 235)
(1000, 240)
(1000, 240)
(1000, 240)
(1000, 235)
(1000, 235)
(1000, 240)


### Convert Reviewers to Index

In [ ]:
final_implicit_score_mat.set_index(final_implicit_score_mat.columns[0], inplace=True)
final_item_cf_score_mat.set_index(final_item_cf_score_mat.columns[0], inplace=True)
final_lightgcn_score_mat.set_index(final_lightgcn_score_mat.columns[0], inplace=True)
final_ncf_score_mat.set_index(final_ncf_score_mat.columns[0], inplace=True)
final_jobs_score_mat.set_index(final_jobs_score_mat.columns[0], inplace=True)
final_cb_score_mat.set_index(final_cb_score_mat.columns[0], inplace=True)
final_user_cf_score_mat.set_index(final_user_cf_score_mat.columns[0], inplace=True)


### Add back missing columns

In [ ]:
missing_course_implicit = list(set(final_ncf_score_mat.columns) - set(final_implicit_score_mat.columns))
missing_course_jobs = list(set(final_ncf_score_mat.columns) - set(final_jobs_score_mat.columns))
missing_course_cb = list(set(final_ncf_score_mat.columns) - set(final_cb_score_mat.columns))
missing_course_user_cf = list(set(final_ncf_score_mat.columns) - set(final_user_cf_score_mat.columns))

In [ ]:
for course in missing_course_implicit:
    final_implicit_score_mat[course] = 0

for course in missing_course_jobs:
    final_jobs_score_mat[course] = 0

for course in missing_course_cb:
    final_cb_score_mat[course] = 0

for course in missing_course_user_cf:
    final_user_cf_score_mat[course] = 0

### Weights to try

In [ ]:
weights = {
    "implicit": [1/7, 1/12, 0.282],
    "ncf": [1/7, 1/12, 0.087],
    "lightgcn": [1/7, 1/12, 0.158],
    "jobs": [1/7, 1/2, 0.057],
    "item_cf": [1/7, 1/12, 0.091],
    "user_cf": [1/7, 1/12, 0.123],
    "cb": [1/7, 1/12, 0.203]
}

### Evaluate the function

In [ ]:
def evaluate_recommendations(recommendations, actual):
    """
    Evaluate recommendations using precision and recall for named reviewers.

    :param recommendations: Pandas Series where index is reviewer names and values are lists of recommended courses.
    :param actual: Pandas Series where index is reviewer names and values are lists of actual courses.
    :returns: Tuple containing average precision and recall for all reviewers.
    """
    precision_list = []
    recall_list = []

    # Iterate over each reviewer's recommendations and actual courses
    for reviewer, recs in recommendations.items():
        acts = actual.get(reviewer, [])
        true_positives = set(recs) & set(acts)

        precision = len(true_positives) / len(recs) if recs else 0
        recall = len(true_positives) / len(acts) if acts else 0

        precision_list.append(precision)
        recall_list.append(recall)

    average_precision = sum(precision_list) / len(precision_list) if precision_list else 0
    average_recall = sum(recall_list) / len(recall_list) if recall_list else 0

    f1 = 2 * (average_precision * average_recall) / (average_precision + average_recall) if average_precision + average_recall else 0

    return average_precision, average_recall, f1

### Evaluate the model

In [ ]:
final = pd.DataFrame(index=final_implicit_score_mat.index, columns=final_implicit_score_mat.columns)
courses_taken = train.groupby('Reviewer')['Course Name'].agg(list)
user_courses_map = courses_taken.to_dict()
best_weight_index = 0
best_f1 = 0
best_precision = 0
best_recall = 0

actual_courses = eval.groupby('Reviewer')['Course Name'].agg(list)

for i in range(3):
  for reviewer in final_implicit_score_mat.index:
    for course in final_implicit_score_mat.columns:
      final.at[reviewer, course] = list(weights.values())[0][i] * final_implicit_score_mat.at[reviewer, course]+ list(weights.values())[1][i] * final_ncf_score_mat.at[reviewer, course]+ list(weights.values())[2][i] * final_lightgcn_score_mat.at[reviewer, course] + list(weights.values())[3][i] * final_jobs_score_mat.at[reviewer, course] + list(weights.values())[4][i] * final_item_cf_score_mat.at[reviewer, course] + list(weights.values())[5][i] * final_user_cf_score_mat.at[reviewer, course] + list(weights.values())[6][i] * final_cb_score_mat.at[reviewer, course]

  for reviewer, courses in user_courses_map.items():
    if reviewer in final.index:
        for course in courses:
            if course in final.columns:
                final.at[reviewer, course] = 0
  final = final.astype(float)
  top_n_recs =  final.apply(lambda x: x.nlargest(10).index.tolist(), axis=1)
  precision, recall, f1 = evaluate_recommendations(top_n_recs, actual_courses)
  if f1 > best_f1:
    best_f1 = f1
    best_precision = precision
    best_recall = recall
    best_weight_index = i

best_weights = {key: values[best_weight_index] for key, values in weights.items()}
print("Best Weights (Evaluation): ", best_weights)
print("Best F1 Score (Evaluation): ", best_f1)
print("Best Precision (Evaluation): ", best_precision)
print("Best Recall (Evaluation): ", best_recall)

Best Weights (Evaluation):  {'implicit': 0.282, 'ncf': 0.087, 'lightgcn': 0.158, 'jobs': 0.057, 'item_cf': 0.091, 'user_cf': 0.123, 'cb': 0.203}
Best F1 Score (Evaluation):  0.06651962356221722
Best Precision (Evaluation):  0.03680000000000025
Best Recall (Evaluation):  0.3457333333333334


In [ ]:
final

,neural networks and deep learning,"improving deep neural networks: hyperparameter tuning, regularization and optimization",sequence models,what is data science?,programming for everybody getting started with python,supply chain principles,using python to interact with the operating system,structuring machine learning projects,the bits and bytes of computer networking,introduction to programming with matlab,...,mastering final cut pro,natural language processing with attention models,getting started with azure,web application technologies and django,high stakes leadership: leading in times of crisis,write your first novel,fundamentals of machine learning for healthcare,social work practice: advocating social justice and change,how to manage a remote team,fundamentals of finance
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
aadarsh r,0.424725,0.349708,0.309321,0.454433,0.365141,0.274332,0.338834,0.325902,0.370963,0.337844,...,0.232823,0.217833,0.259999,0.244682,0.222658,0.046277,0.0,0.0,0.0,0.035980
aakash j,0.379848,0.331345,0.301703,0.418022,0.518910,0.269626,0.311736,0.272712,0.355283,0.417078,...,0.244083,0.210098,0.250929,0.266990,0.251823,0.032328,0.0,0.0,0.0,0.035385
abbas h m,0.406388,0.343780,0.268918,0.313349,0.476147,0.239292,0.305427,0.314670,0.293026,0.291468,...,0.224694,0.221216,0.232156,0.271702,0.257841,0.015975,0.0,0.0,0.0,0.014970
abdelkader b,0.463186,0.303527,0.304559,0.329618,0.342105,0.276095,0.277548,0.304454,0.307315,0.301157,...,0.241621,0.214348,0.235210,0.246420,0.227253,0.013082,0.0,0.0,0.0,0.020177
abdulrahman a m,0.427012,0.322747,0.324426,0.314299,0.390582,0.294806,0.291605,0.321635,0.327522,0.323275,...,0.262463,0.220593,0.252374,0.255002,0.242587,0.034365,0.0,0.0,0.0,0.032263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zahir k,0.494666,0.420942,0.302786,0.527367,0.437161,0.274456,0.260596,0.392826,0.317474,0.289447,...,0.228180,0.220630,0.238527,0.231489,0.220072,0.018593,0.0,0.0,0.0,0.017656
zarima j,0.444080,0.321288,0.258608,0.302827,0.392711,0.331651,0.334908,0.302783,0.366325,0.313749,...,0.299101,0.210876,0.304389,0.307459,0.303090,0.091229,0.0,0.0,0.0,0.089106
zeeshan z,0.389110,0.316301,0.330385,0.351930,0.380689,0.282845,0.253415,0.326776,0.322809,0.314125,...,0.229350,0.208577,0.231391,0.237647,0.209843,0.021488,0.0,0.0,0.0,0.022864


### Evaluation on Test Set

In [ ]:
final_weights = {
    "implicit": list(weights.values())[0][best_weight_index],
    "ncf": list(weights.values())[1][best_weight_index],
    "lightgcn": list(weights.values())[2][best_weight_index],
    "jobs": list(weights.values())[3][best_weight_index],
    "item_cf": list(weights.values())[4][best_weight_index],
    "user_cf": list(weights.values())[5][best_weight_index],
    "cb": list(weights.values())[6][best_weight_index]
}

final = pd.DataFrame(index=final_implicit_score_mat.index, columns=final_implicit_score_mat.columns)
test_weight_index = 0
test_f1 = 0
test_precision = 0
test_recall = 0
top_n_rec = False
test_courses = test.groupby('Reviewer')['Course Name'].agg(list)

for reviewer in final_implicit_score_mat.index:
  for course in final_implicit_score_mat.columns:
    final.at[reviewer, course] = list(final_weights.values())[0] * final_implicit_score_mat.at[reviewer, course]+ list(final_weights.values())[1] * final_ncf_score_mat.at[reviewer, course]+ list(final_weights.values())[2] * final_lightgcn_score_mat.at[reviewer, course] + list(final_weights.values())[3] * final_jobs_score_mat.at[reviewer, course] + list(final_weights.values())[4] * final_item_cf_score_mat.at[reviewer, course] + list(final_weights.values())[5] * final_user_cf_score_mat.at[reviewer, course] + list(final_weights.values())[6] * final_cb_score_mat.at[reviewer, course]

for reviewer, courses in user_courses_map.items():
  if reviewer in final.index:
      for course in courses:
          if course in final.columns:
              final.at[reviewer, course] = 0
final = final.astype(float)
top_n_recs =  final.apply(lambda x: x.nlargest(10).index.tolist(), axis=1)
precision, recall, f1 = evaluate_recommendations(top_n_recs, test_courses)
test_f1 = f1
test_precision = precision
test_recall = recall
test_weight_index = i

test_weights = {key: values for key, values in final_weights.items()}
print("Best Weights (Test): ", test_weights)
print("Best F1 Score (Test): ", test_f1)
print("Best Precision (Test): ", test_precision)
print("Best Recall (Test): ", test_recall)

Best Weights (Test):  {'implicit': 0.282, 'ncf': 0.087, 'lightgcn': 0.158, 'jobs': 0.057, 'item_cf': 0.091, 'user_cf': 0.123, 'cb': 0.203}
Best F1 Score (Test):  0.04950176429075531
Best Precision (Test):  0.027400000000000115
Best Recall (Test):  0.256


### Sample Recommendations

In [ ]:
sample_user = "kumar k"

sample_user_recs = top_n_recs[sample_user]
print(f"Course taken by {sample_user}:\n{user_courses_map[sample_user]}")
print(f"Top 10 Recommendations for {sample_user}:")

for recc in  sample_user_recs[:10]:
    print(recc)

Course taken by kumar k:
['python data structures', 'using python to access web data', 'introduction to marketing']
Top 10 Recommendations for kumar k:
introduction to financial accounting
programming for everybody getting started with python
excel skills for business: essentials
ai for everyone
fundamentals of graphic design
introduction to data science in python
neural networks and deep learning
introduction to operations management
introduction to corporate finance
what is data science?
